In [1]:
!pip install --quiet lpips


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [2]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from tqdm import tqdm # Opsional, tapi sangat disarankan untuk progress bar
from torch.utils.data import random_split
from math import floor

import os
import shutil

import csv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- import LPIPS ---
try:
    import lpips  # pip install lpips
except ImportError as e:
    raise ImportError("Package 'lpips' belum terpasang. Jalankan: pip install lpips") from e

## Fungsi train, custom dataset

In [3]:
import os, csv, time, shutil
import torch
from tqdm import tqdm
import pandas as pd
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
import lpips  # pip install lpips

def _read_last_epoch(csv_path):
    if not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0:
        return 0
    try:
        df = pd.read_csv(csv_path)
        if 'epoch' in df.columns and pd.api.types.is_numeric_dtype(df['epoch']):
            ep = df['epoch'].dropna()
            return int(ep.max()) if len(ep) else 0
        return 0
    except Exception:
        return 0

def _prepare_csv(seed_csv, path_write):
    """Jika path_write belum ada dan seed_csv ada, salin ke working."""
    os.makedirs(os.path.dirname(path_write), exist_ok=True)
    if (seed_csv is not None and os.path.exists(seed_csv)) and (not os.path.exists(path_write)):
        shutil.copyfile(seed_csv, path_write)

def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs,
    save_dir="model_checkpoints",
    seed_csv="/kaggle/input/history1/histor.csv",      # <-- sumber read-only (opsional)
    path_write="/kaggle/working/histor.csv",           # <-- tujuan APPEND (writable)
    continue_global_epoch=True
):
    # siapkan folder writable dan CSV tujuan
    os.makedirs(save_dir, exist_ok=True)
    _prepare_csv(seed_csv, path_write)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # metrics
    psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
    ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
    lpips_fn = lpips.LPIPS(net='vgg').to(device).eval()

    def to_minus1_1(x): return x*2.0 - 1.0

    history = {'train_loss': [], 'val_loss': [], 'val_psnr': [], 'val_ssim': [], 'val_lpips': []}

    run_id = time.strftime("%Y-%m-%d_%H-%M-%S")
    start_epoch_global = _read_last_epoch(path_write) if continue_global_epoch else 0
    print(f"Mulai Training di {device} | run_id={run_id} | start_epoch_global={start_epoch_global}")
    print(f"CSV tulis: {path_write}")

    for e in range(num_epochs):
        epoch_in_run = e + 1
        epoch_global = start_epoch_global + epoch_in_run

        # ===== TRAIN =====
        model.train()
        running_train_loss = 0.0
        for lr_images, hr_images in tqdm(train_loader, desc=f"Epoch {epoch_in_run}/{num_epochs} [Train]"):
            lr_images = lr_images.to(device); hr_images = hr_images.to(device)
            optimizer.zero_grad()
            pred = model(lr_images)
            loss = criterion(pred, hr_images)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * lr_images.size(0)
        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_train_loss)

        # ===== VAL =====
        model.eval()
        running_val_loss = 0.0
        psnr_metric.reset(); ssim_metric.reset()
        lpips_sum = 0.0
        with torch.no_grad():
            for lr_images, hr_images in tqdm(val_loader, desc=f"Epoch {epoch_in_run}/{num_epochs} [Val]"):
                lr_images = lr_images.to(device); hr_images = hr_images.to(device)
                pred = model(lr_images)
                vloss = criterion(pred, hr_images)
                running_val_loss += vloss.item() * lr_images.size(0)

                psnr_metric.update(pred, hr_images)
                ssim_metric.update(pred, hr_images)

                lp = lpips_fn(to_minus1_1(pred), to_minus1_1(hr_images)).mean().item()
                lpips_sum += lp * lr_images.size(0)

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_psnr = psnr_metric.compute().item()
        epoch_ssim = ssim_metric.compute().item()
        epoch_lpips = lpips_sum / len(val_loader.dataset)

        history['val_loss'].append(epoch_val_loss)
        history['val_psnr'].append(epoch_psnr)
        history['val_ssim'].append(epoch_ssim)
        history['val_lpips'].append(epoch_lpips)

        print(f"[RESULT] e{epoch_in_run} | val_loss={epoch_val_loss:.6f} | PSNR={epoch_psnr:.4f} | SSIM={epoch_ssim:.4f} | LPIPS={epoch_lpips:.4f}")

        # ===== SAVE CHECKPOINT =====
        ckpt_name = f"model_{run_id}_e{epoch_in_run}.pth"
        save_path = os.path.join(save_dir, ckpt_name)
        torch.save(model.state_dict(), save_path)
        print(f"[SAVE] {save_path}")

        # ===== APPEND ke CSV (SELALU ke path_write di /kaggle/working) =====
        is_new = not os.path.exists(path_write) or os.path.getsize(path_write) == 0
        with open(path_write, "a", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(
                f,
                fieldnames=[
                    "epoch","epoch_in_run","run_id",
                    "train_loss","val_loss","val_psnr","val_ssim","val_lpips","nama_model"
                ]
            )
            if is_new:
                w.writeheader()
            w.writerow({
                "epoch":        epoch_global,
                "epoch_in_run": epoch_in_run,
                "run_id":       run_id,
                "train_loss":   float(epoch_train_loss),
                "val_loss":     float(epoch_val_loss),
                "val_psnr":     float(epoch_psnr),
                "val_ssim":     float(epoch_ssim),
                "val_lpips":    float(epoch_lpips),
                "nama_model":   ckpt_name
            })

    print("Training Selesai.")
    return history


In [4]:
class SuperResolutionDataset(Dataset):
    """
    Dataset kustom untuk super-resolution.
    Akan memuat gambar 256x256 (HR) dan secara otomatis
    membuat versi 128x128 (LR) sebagai input.
    """
    def __init__(self, image_folder):
        self.root_dir = image_folder
        # Dapatkan daftar semua nama file gambar di folder
        self.image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        # Definisikan transform untuk HR (hanya konversi ke Tensor)
        self.hr_transform = T.ToTensor()
        
        # Definisikan transform untuk LR (Resize ke 128x128 + konversi ke Tensor)
        self.lr_transform = T.Compose([
            T.Resize((128, 128), interpolation=T.InterpolationMode.BICUBIC),
            T.ToTensor()
        ])

    def __len__(self):
        # Mengembalikan jumlah total gambar
        return len(self.image_files)

    def __getitem__(self, idx):
        # 1. Dapatkan path lengkap ke satu gambar
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        
        # 2. Buka gambar HR (256x256) menggunakan PIL
        #    Gunakan .convert('RGB') untuk memastikan gambar punya 3 channel
        hr_image = Image.open(img_path).convert('RGB')
        
        # 3. Buat tensor HR dan LR dari gambar yang sama
        lr_tensor = self.lr_transform(hr_image)
        hr_tensor = self.hr_transform(hr_image)
        
        # 4. Kembalikan pasangan (input, target)
        return lr_tensor, hr_tensor

## ARSITEKTUR

In [5]:
class blocks (nn.Module):

    def __init__(self, k_feature, N,H,W,C,j, head):
        super().__init__()
        self.j = j
        self.N, self.C, self.H, self.W = N,C,H,W
        self.k_feature = k_feature
        self.head = head
        self.dk = self.k_feature//self.head # dmodel = k_feature
        self.n_win = self.H//self.j * self.W//self.j
        self.rasio_ekspansi = 4
    
    # ============================ body Rir =====================================
        # normalisasi
        self.layernorm1 = nn.LayerNorm(normalized_shape=[self.k_feature])
        self.layernorm2 = nn.LayerNorm(normalized_shape=[self.k_feature])

        # Linear projections
        self.wq = nn.Parameter(torch.empty(self.k_feature, self.k_feature))
        self.wk = nn.Parameter(torch.empty(self.k_feature, self.k_feature))
        self.wv = nn.Parameter(torch.empty(self.k_feature, self.k_feature))
        self.wo = nn.Parameter(torch.empty(self.k_feature, self.k_feature))
        self.bt = nn.Parameter(torch.zeros(self.head, self.j*self.j, self.j*self.j))

        # MLP
        self.w1 = nn.Parameter(torch.empty(self.k_feature, self.k_feature * self.rasio_ekspansi))
        self.b1 = nn.Parameter(torch.zeros([self.k_feature*self.rasio_ekspansi]))
        self.w2 = nn.Parameter(torch.empty(self.k_feature*self.rasio_ekspansi, self.k_feature))
        self.b2 = nn.Parameter(torch.zeros([self.k_feature]))

        # Panggil fungsi inisialisasi
        self.init_weights()

    def init_weights(self):
        # === Xavier initialization untuk layer linear ===
        nn.init.xavier_uniform_(self.wq)
        nn.init.xavier_uniform_(self.wk)
        nn.init.xavier_uniform_(self.wv)
        nn.init.xavier_uniform_(self.wo)
        nn.init.xavier_uniform_(self.w1)
        nn.init.xavier_uniform_(self.w2)

        # Bias nol sudah oke (default)
        nn.init.zeros_(self.b1)
        nn.init.zeros_(self.b2)

        # Bisa juga He init jika ingin lebih cocok untuk ReLU:
        # nn.init.kaiming_normal_(self.w1, nonlinearity='relu')
        # nn.init.kaiming_normal_(self.w2, nonlinearity='relu')


    def forward(self, x):

        
        # input kedala fungsi ini x [8,96,128,128]
        
        # =================================================--=======================================================
        # ============================================== MSA =======================================================
        # =================================================--=======================================================
        
        # menukra ukuran dimensi (N,C,H,W) => (N,H,W,C)
        xp = x.permute(0,2,3,1)
        
        # normalisasi input
       
        xln1 = self.layernorm1(xp)
        N_dyn, H_dyn, W_dyn, C_dyn = xln1.shape

        # window partition. 
        # reshape (N,H,W,C) => ((N X n_win),j,j,k_feature)
        # 1.view potong H dan W menjadi (numwin_H, j) dan (NumWin_W, j)
        xr = xln1.view(N_dyn, H_dyn//self.j, self.j, W_dyn//self.j, self.j, C_dyn)
        
        # 2. permute [N, H//J, W//J, j,j, k_feature]
        xwin = xr.permute(0,1,3,2,4,5).contiguous()

        # 3.view gabugnkan semua jendela kedalam dimenci batch
        # [N * num_wind * numwind , j,j, C]
        xwin = xwin.view(-1,self.j,self.j,C_dyn)
        # xr = xln1.reshape(self.N * self.n_win, self.j, self.j, self.k_feature)

        # flatten ((N X n_win),j,j,C) => ((N X n_win),j*j,C)  
        xf = xwin.reshape(-1, self.j * self.j, self.k_feature)

        # self attention
        # mencari Q,K,V
        Q = xf @ self.wq
        K = xf @ self.wk
        V = xf @ self.wv

        # split heads mengubah ukuran Q,K,V [2048,64,96] => menjadi [2048,64,6,16]
        Q = Q.reshape((-1, self.j * self.j, self.head, self.k_feature//self.head))
        K = K.reshape((-1, self.j * self.j, self.head, self.k_feature//self.head))
        V = V.reshape((-1, self.j * self.j, self.head, self.k_feature//self.head))

        # permute dari bentuk [2048,64,6,16] => [2048,6,64,16]
        Q = Q.permute(0,2,1,3)
        K = K.permute(0,2,1,3)
        V = V.permute(0,2,1,3)

        # SCALED dot product attention
        score = Q.matmul(K.transpose(-2,-1))
        scaling = score/(self.dk ** 0.5)
        B = scaling + self.bt
        P = F.softmax(B, dim=-1)
        O = P @ V

        #permute dari [2048,6,64,16] => [2048,64,6,16]
        O = O.permute(0,2,1,3)
        
        #reshape menggabungkan head dari [2048,64,6,16] => [2048,64,96]
        O = O.reshape(-1, self.j * self.j, self.k_feature )

        # MSA_out
        MSA_out = O @ self.wo

        # mengubah kembali ke bentuk input awal [N,C,H,W]
        # 1. Reshape ke [256, 8, 8, 96]
        MSA_out = MSA_out.view(-1, self.j, self.j, self.k_feature) 
        
        # 2. Reshape ke [N_dyn, NumWin_H, NumWin_W, j, j, C]
        MSA_out = MSA_out.view(N_dyn, H_dyn // self.j, W_dyn // self.j, self.j, self.j, self.k_feature)
        
        # 3. Permute
        MSA_out = MSA_out.permute(0, 1, 3, 2, 4, 5).contiguous()
        
        # 4. View ke [N_dyn, H_dyn, W_dyn, C]
        MSA_out = MSA_out.view(N_dyn, H_dyn, W_dyn, self.k_feature)

        # # reshape dari ukuran [2048,64,96] => [2048,8,8,96]
        # MSA_out = MSA_out.reshape(self.N * self.n_win,self.j, self.j, self.k_feature)

        # # reshape dari ukuran [2048,8,8,96] => [8,256,8,8,96]
        # MSA_out = MSA_out.reshape(self.N, self.n_win, self.j, self.j, self.k_feature)

        # # reshape dari ukuran [8,256,8,8,96] => [8,128,128,96]
        # MSA_out = MSA_out.reshape(self.N, self.H, self.W, self.k_feature)

        # permute untuk mengembalikan ke bentuk awal N,C,H,W
        MSA_out = MSA_out.permute(0,-1,1,2)

        #============ RESIDUAL =======================
        xmsa = x + MSA_out # [N,k_feature, H,W] [8.96,128,128]

        # =================================================--=======================================================
        # ============================================== MLP =======================================================
        # =================================================--=======================================================

        # input x [N,k_feature, H,W] [8.96,128,128]
        x2 = xmsa.clone()
       # menukra ukuran dimensi (N,C,H,W) => (N,H,W,C)
        xp = x2.permute(0,2,3,1)
    
        # normalisasi input
        xln = self.layernorm2(xp)

        # Multi Layer Perceptron 
        # layer1 (8,128,128,96) => (8,128,128,384)
        x2 = xln @ self.w1 + self.b1
        # aktivasi
        x2 = F.gelu(x2)

        # layer2 (8,128,128,384) => (8,128,128,96)
        x2 = x2 @ self.w2 + self.b2

        # permutasi
        x2 = x2.permute(0,3,1,2)

        # residual
        x_out = x2 + xmsa
        return x_out

class group(nn.Module):
    def __init__(self, k_feature, N,H,W,C,j, head, j_blocks):
        super().__init__()
        self.j = j
        self.N, self.C, self.H, self.W = N,C,H,W
        self.k_feature = k_feature
        self.head = head
        self.dk = self.k_feature//self.head # dmodel = k_feature
        self.n_win = self.H//self.j * self.W/self.j
        
        self.bloks = nn.ModuleList([
            blocks(k_feature=self.k_feature, N=self.N,H=self.H,W=self.W,C=self.C,j=self.j, head = self.head) for _ in range(j_blocks)
        ])

    def forward(self, x):
        xin = x.clone()
        for blk in self.bloks:
            x = blk(x)
        # residual \
        x_out = xin + x
        return x_out

    

class RIR_interpolation(nn.Module):

    def __init__(self, N,C,H,W,  k_feature, j_group, j_blocks):
        super().__init__()
        self.N, self.C, self.H, self.W = N,C,H,W
        # self.groups = groups
        self.blocks = j_blocks
        self.j = 8  # ukuran window
        self.k_feature = k_feature
        self.head = 6
        self.dk = self.k_feature//self.head # dmodel = k_feature
        self.j_group = j_group
        
        self.n_win = self.H//self.j * self.W//self.j
        
        # [out_channels, in_channels, kernel, kernel]

        # ========================== ekstraksi fitur ================================
        self.w0 = nn.Parameter(torch.empty(self.k_feature, 3, 3, 3))
        self.w1 = nn.Parameter(torch.empty(self.k_feature * 4, k_feature, 3, 3))
        self.w2 = nn.Parameter(torch.empty(3, self.k_feature, 3, 3))
        self.b0 = nn.Parameter(torch.zeros(self.k_feature))
        self.b1 = nn.Parameter(torch.zeros(self.k_feature * 4))
        self.b2 = nn.Parameter(torch.zeros(3))


        # body 
        self.body = nn.ModuleList([
            group(self.k_feature, self.N, self.H, self.W, self.C, self.j, self.head, 6) for _ in range(self.j_group)
        ])
        
        
        
        self.pixshuffle = nn.PixelShuffle(upscale_factor=2)
        self.init_weights()
    

    def init_weights(self):
        # He initialization untuk convolution
        nn.init.kaiming_normal_(self.w0, mode='fan_out', nonlinearity='relu')
        nn.init.kaiming_normal_(self.w1, mode='fan_out', nonlinearity='relu')
        nn.init.kaiming_normal_(self.w2, mode='fan_out', nonlinearity='relu')
        nn.init.zeros_(self.b0)
        nn.init.zeros_(self.b1)
        nn.init.zeros_(self.b2)

    
        

    def forward(self,x):

        # =================Ekstraksi Fitur===========================
        # input awal X [N,C,H,W] [8,3,128,128]
        d = F.conv2d(x, self.w0, bias=self.b0, padding=1)
        d = F.relu(d) # [8,96,128,128]
        # output = X = [N,k_feature,H,W] [8,96,128,128]

        
        # ================== body rir ==============================
        #  input awal X [N,k_featur,H,W] = [8,96,128,128]
        
        # b = self.body.forward(d)
        din = d.clone()
        for grp in self.body:
            d = grp(d)

        # residual
        b = din + d
            

        # output sama dengan input X = X [N,k_featur,H,W] = [8,96,128,128]

        
        # =============== Upsampler/pixxel shuffle ==================
        # input awal X [N,k_featur,H,W] = [8,96,128,128]
        d = F.conv2d(b, self.w1, bias=self.b1, padding=1)
        d = F.relu(d)
        o = self.pixshuffle(d)

        # Rekonstruksi
        xsr = F.conv2d(o, self.w2, bias=self.b2, padding=1)
        # output X [N.C,H*2,W*2]  [8,3,256,256]
        
        return xsr

## Load Model

In [6]:
def load_model(model_architecture, path_hist):
    """
    Memuat bobot (weights) yang tersimpan ke dalam arsitektur model.

    Args:
        model_architecture (nn.Module): Objek model yang arsitekturnya SAMA
                                        (misal: Cnn_interpolation(k_feature=64)).
        path_to_weights (str): Path ke file .pth yang disimpan.

    Returns:
        nn.Module: Model dengan bobot yang sudah dimuat.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # baca file csv
    df = pd.read_csv(path_hist)
    path_to_weights = "/kaggle/input/mod/pytorch/lpips1/1/"+df['nama_model'].iloc[-1]
    
    
    # Muat state_dict dari file
    # map_location=device memastikan bobot dimuat ke device yang benar
    state_dict = torch.load(path_to_weights, map_location=device)
    
    # Masukkan state_dict (bobot) ke dalam arsitektur model
    model_architecture.load_state_dict(state_dict)
    
    # Pindahkan model ke device
    model_architecture.to(device)
    
    # PENTING: Set model ke mode evaluasi/inferensi
    # Ini menonaktifkan layer seperti Dropout atau BatchNorm (jika ada)
    model_architecture.eval() 
    
    print(f"Model berhasil dimuat dari {path_to_weights}")
    return model_architecture

In [7]:
import tkinter as tk
from tkinter import filedialog

def pilih_file_weight():
    """
    Membuka file explorer untuk memilih file .pth.
    """
    root = tk.Tk()
    root.withdraw()  # sembunyikan jendela utama Tkinter
    file_path = filedialog.askopenfilename(
        title="Pilih file weight (.pth)",
        filetypes=[("PyTorch model", "*.pth"), ("Semua file", "*.*")]
    )
    return file_path


# ==== Contoh penggunaan ====
# Misalnya kamu sudah punya arsitektur model
# model = Cnn_interpolation(k_feature=64)


## Load Dataset

In [8]:
import kagglehub

# Download latest version
PATH = kagglehub.dataset_download("mcparadip/anime-faces-waifu2x")

# # print("Path to dataset files:", path)
# PATH = "/home/kanza/Dokumen/file/D2L/transformer/anim/archive" # <-- GANTI INI


## Training

### Part

In [9]:
# import torch
# from torch.utils.data import DataLoader, Subset, random_split

# # Tentukan path ke folder gambar Anda
# PATH_KE_FOLDER_GAMBAR = PATH # <-- GANTI INI
# BATCH_SIZE = 1
# NUM_EPOCHS = 1
# K_FEATURE = 96

# # --- MODIFIKASI DIMULAI DI SINI ---

# # 1. Buat dataset Lengkap
# full_dataset = SuperResolutionDataset(image_folder=PATH_KE_FOLDER_GAMBAR)
# print(f"Ukuran dataset penuh: {len(full_dataset)}")

# # 2. Tentukan jumlah gambar yang ingin digunakan (misal, 100)
# num_images_to_use = 2
# indices = list(range(num_images_to_use))

# # 3. Buat SUBSET dari dataset penuh
# subset_dataset = Subset(full_dataset, indices)
# print(f"Ukuran subset yang akan digunakan: {len(subset_dataset)}")

# # 4. Tentukan ukuran split (sekarang berdasarkan subset_dataset)
# train_size = int(0.8 * len(subset_dataset)) # Ini akan menjadi 80
# val_size = len(subset_dataset) - train_size   # Ini akan menjadi 20

# # 5. Lakukan split pada SUBSET tersebut
# train_dataset, val_dataset = random_split(subset_dataset, [train_size, val_size])

# print(f"Ukuran data training: {len(train_dataset)}")
# print(f"Ukuran data validasi: {len(val_dataset)}")

# # --- MODIFIKASI SELESAI ---

# # 6. Buat data loader train dan val (kode ini tetap sama)
# train_loader = DataLoader(
#     train_dataset, 
#     batch_size=BATCH_SIZE, 
#     shuffle=True, 
#     num_workers=4,
#     pin_memory=True
# )

# val_loader = DataLoader(
#     val_dataset,
#     batch_size=BATCH_SIZE,
#     shuffle=False, # Tidak perlu di-shuffle untuk validasi
#     num_workers=4,
#     pin_memory=True
# )

### Full

In [10]:
# Tentukan path ke folder gambar Anda
PATH_KE_FOLDER_GAMBAR = PATH # <-- GANTI INI
BATCH_SIZE = 1

K_FEATURE = 96

# 1. Buat dataset Lengkap
full_dataset =  SuperResolutionDataset(image_folder=PATH_KE_FOLDER_GAMBAR)
print(len(full_dataset))

# 2. Tentukan ukuran split 
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# 3.Lakukan split
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

print(f"Ukuran data training: {len(train_dataset)}")
print(f"Ukuran data validasi: {len(val_dataset)}")

# 4. buat data loader train dan val
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=1,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, # Tidak perlu di-shuffle untuk validasi
    num_workers=1,
    pin_memory=True
)


21551
Ukuran data training: 17240
Ukuran data validasi: 4311


## Training

In [11]:
# NUM_EPOCHS = 1
# # ================================================== inisialisasi model ==========================================
# model = RIR_interpolation(
#     N=1,
#     C=3,
#     H=128,
#     W=128,
#     k_feature=96,
#     j_group=6,
#     j_blocks=6)

# model = model.to(device)           # ⬅️ pindahkan SEMUA parameter model ke GPU

# # tentukan loss function
# criterion = nn.L1Loss()

# # menentukan optimizer 
# optimizer = optim.Adam(model.parameters(),lr=0.0001)


# # ================================= Trainig ===================================================================
# # 6. Mulai Training dan TANGKAP output history
# history = train_model(
#     model=model,
#     train_loader=train_loader,
#     val_loader=val_loader,  # Berikan val_loader di sini
#     criterion=criterion,
#     optimizer=optimizer,
#     num_epochs=NUM_EPOCHS,
#     save_dir="model_checkpoints",
#     continue_global_epoch=False
# )

# # 7. Tampilkan hasil history
# print("\n--- Hasil History Training ---")
# import json
# print(json.dumps(history, indent=2))



In [12]:
import pandas as pd
df = pd.read_csv('/kaggle/input/history1/histor.csv')
df

,epoch,epoch_in_run,run_id,train_loss,val_loss,val_psnr,val_ssim,val_lpips,nama_model
0,1,1,2025-11-12_07-02-58,1.526776,0.114601,16.121893,0.324323,0.540916,model_2025-11-12_07-02-58_e1.pth


## Fine Tuning

In [13]:
model = RIR_interpolation(
    N=1,
    C=3,
    H=128,
    W=128,
    k_feature=96,
    j_group=6,
    j_blocks=6)


load_model(model, '/kaggle/input/history1/histor.csv')

model = model.to(device)           # ⬅️ pindahkan SEMUA parameter model ke GPU

# tentukan loss function
criterion = nn.L1Loss()

# menentukan optimizer 
optimizer = optim.Adam(model.parameters(),lr=0.0001)


# ================================= Trainig ===================================================================
# 6. Mulai Training dan TANGKAP output history
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=5,
    save_dir="/kaggle/working/model_checkpoints",
    seed_csv="/kaggle/input/history1/histor.csv",     # baca dari input (opsional)
    path_write="/kaggle/working/histor.csv"           # tulis/append ke working
)


# 7. Tampilkan hasil history
print("\n--- Hasil History Training ---")
import json
print(json.dumps(history, indent=2))



Model berhasil dimuat dari /kaggle/input/mod/pytorch/lpips1/1/model_2025-11-12_07-02-58_e1.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 211MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth
Mulai Training di cuda | run_id=2025-11-12_09-22-03 | start_epoch_global=1
CSV tulis: /kaggle/working/histor.csv


Epoch 1/5 [Val]: 100%|██████████| 4311/4311 [06:41<00:00, 10.74it/s]


[RESULT] e1 | val_loss=0.031248 | PSNR=26.7208 | SSIM=0.8509 | LPIPS=0.2204
[SAVE] /kaggle/working/model_checkpoints/model_2025-11-12_09-22-03_e1.pth


...

[RESULT] e3 | val_loss=0.261625 | PSNR=10.4710 | SSIM=0.3955 | LPIPS=0.7076
[SAVE] /kaggle/working/model_checkpoints/model_2025-11-12_09-22-03_e3.pth


Epoch 4/5 [Val]: 100%|██████████| 4311/4311 [06:40<00:00, 10.76it/s]


[RESULT] e4 | val_loss=0.259247 | PSNR=10.2452 | SSIM=0.3974 | LPIPS=0.6947
[SAVE] /kaggle/working/model_checkpoints/model_2025-11-12_09-22-03_e4.pth


Epoch 5/5 [Val]: 100%|██████████| 4311/4311 [06:39<00:00, 10.79it/s]

[RESULT] e5 | val_loss=0.259258 | PSNR=10.2848 | SSIM=0.3971 | LPIPS=0.6957
[SAVE] /kaggle/working/model_checkpoints/model_2025-11-12_09-22-03_e5.pth
Training Selesai.

--- Hasil History Training ---
{
  "train_loss": [
    0.27105162701949803,
    0.1739790973905582,
    0.21526851209859404,
    0.26107006108262976,
    0.26029868522645977
  ],
  "val_loss": [
    0.031248132923561357,
    0.06386741599432279,
    0.26162478592176774,
    0.25924717349924253,
    0.25925772300979927
  ],
  "val_psnr": [
    26.72084617614746,
    21.080400466918945,
    10.47098445892334,
    10.245247840881348,
    10.284793853759766
  ],
  "val_ssim": [
    0.8508930802345276,
    0.5806698799133301,
    0.3954969346523285,
    0.39738908410072327,
    0.3971056342124939
  ],
  "val_lpips": [
    0.22044456999544146,
    0.4095101043965639,
    0.7075588993919999,
    0.6946855832268926,
    0.6957312956870612
  ]
}


In [14]:
import pandas as pd
df = pd.read_csv('/kaggle/working/histor.csv')
df

,epoch,epoch_in_run,run_id,train_loss,val_loss,val_psnr,val_ssim,val_lpips,nama_model
0,1,1,2025-11-12_07-02-58,1.526776,0.114601,16.121893,0.324323,0.540916,model_2025-11-12_07-02-58_e1.pth
1,2,1,2025-11-12_09-22-03,0.271052,0.031248,26.720846,0.850893,0.220445,model_2025-11-12_09-22-03_e1.pth
2,3,2,2025-11-12_09-22-03,0.173979,0.063867,21.080400,0.580670,0.409510,model_2025-11-12_09-22-03_e2.pth
3,4,3,2025-11-12_09-22-03,0.215269,0.261625,10.470984,0.395497,0.707559,model_2025-11-12_09-22-03_e3.pth
4,5,4,2025-11-12_09-22-03,0.261070,0.259247,10.245248,0.397389,0.694686,model_2025-11-12_09-22-03_e4.pth
5,6,5,2025-11-12_09-22-03,0.260299,0.259258,10.284794,0.397106,0.695731,model_2025-11-12_09-22-03_e5.pth
